# High Dimensional Weighted Gene Correlation Network Analysis (hdWGCNA)

**Authorship:**
Adam Klie, *08/24/2022*
***
**Description:**
Notebook to run a WGCNA on stimulated pancreatic islet multiome data (well just the RNA)
***

In [1]:
# single-cell analysis package
library(Seurat)

# plotting and data science packages
library(tidyverse)
library(cowplot)
library(patchwork)

# co-expression network analysis packages:
library(WGCNA)
library(hdWGCNA)

# network analysis & visualization package:
library(igraph)

# using the cowplot theme for ggplot
theme_set(theme_cowplot())

# set random seed for reproducibility
set.seed(12345)

Attaching SeuratObject

Attaching sp

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘patchwork’


The following object is masked from ‘package:cowplot’:

    align_plots


Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor



Attaching package: ‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:purrr’:

    compos

In [3]:
# Parameters
CELL.TYPES <- "beta"  # Meaning α, β, δ in this context
ASSAY <- "RNA"
NORMALIZATION <- "ND"
NN <- 25
GENES <- "all"

In [4]:
# File names for saving
NAME=paste0(CELL.TYPES, "cells_", GENES, "genes_", NORMALIZATION, "_", NN, "neighbors")
OUT= file.path("./network_analysis", NAME)
OUT

[1] "./network_analysis/betacells_allgenes_ND_25neighbors"

In [5]:
# load the preprocessed RDS from 1_network_construction.ipynb
seurat_obj <- readRDS(sprintf('network_construction/%s_hdWGCNA.rds', NAME))

In [6]:
ModuleNetworkPlot(
    seurat_obj,
    outdir="./network_analysis"
)

Writing output files to ./network_analysis

Selecting by kME_betacells_allgenes_ND_25neighbors-M1
Selecting by kME_betacells_allgenes_ND_25neighbors-M2
Selecting by kME_betacells_allgenes_ND_25neighbors-M3
Selecting by kME_betacells_allgenes_ND_25neighbors-M4
Selecting by kME_betacells_allgenes_ND_25neighbors-M5


[1] "betacells_allgenes_ND_25neighbors-M1"
[1] "betacells_allgenes_ND_25neighbors-M2"
[1] "betacells_allgenes_ND_25neighbors-M3"
[1] "betacells_allgenes_ND_25neighbors-M4"
[1] "betacells_allgenes_ND_25neighbors-M5"


In [7]:
# hubgene network
options(repr.plot.width=12, repr.plot.height=12)
png(sprintf("%s_HubGeneNetworkPlot.png", OUT), widt=1600, height=1600)
HubGeneNetworkPlot(
  seurat_obj,
  n_hubs = 10, n_other=5,
  edge_prop = 0.75,
  mods = 'all'
)
dev.off()

Selecting by kME_betacells_allgenes_ND_25neighbors-M1
Selecting by kME_betacells_allgenes_ND_25neighbors-M2
Selecting by kME_betacells_allgenes_ND_25neighbors-M3
Selecting by kME_betacells_allgenes_ND_25neighbors-M4
Selecting by kME_betacells_allgenes_ND_25neighbors-M5



                                grey betacells_allgenes_ND_25neighbors-M1 
                                   5                                   15 
betacells_allgenes_ND_25neighbors-M2 betacells_allgenes_ND_25neighbors-M3 
                                  15                                   15 
betacells_allgenes_ND_25neighbors-M4 betacells_allgenes_ND_25neighbors-M5 
                                  15                                   15 
[1] "turquoise" "grey90"    "blue"      "brown"     "green"     "yellow"   
[1] "here"
[1] 1554    4


png 
  2

In [8]:
seurat_obj <- RunModuleUMAP(
  seurat_obj,
  n_hubs = 10, # number of hub genes to include for the UMAP embedding
  n_neighbors=15, # neighbors parameter for UMAP
  min_dist=0.1 # min distance between points in UMAP space
)

Selecting by kME_betacells_allgenes_ND_25neighbors-M1
Selecting by kME_betacells_allgenes_ND_25neighbors-M2
Selecting by kME_betacells_allgenes_ND_25neighbors-M3
Selecting by kME_betacells_allgenes_ND_25neighbors-M4
Selecting by kME_betacells_allgenes_ND_25neighbors-M5


In [9]:
# get the hub gene UMAP table from the seurat object
umap_df <- GetModuleUMAP(seurat_obj)

In [10]:
# plot with ggplot
options(repr.plot.width=12, repr.plot.height=12)
png(sprintf("%s_HubGeneUMAPPlot.png", OUT), widt=1600, height=1600)
ggplot(umap_df, aes(x=UMAP1, y=UMAP2)) +
  geom_point(
   color=umap_df$color, # color each point by WGCNA module
   size=umap_df$kME*2 # size of each point based on intramodular connectivity
  ) +
  umap_theme()
dev.off()

png 
  2

In [11]:
options(repr.plot.width=12, repr.plot.height=12)
png(sprintf("%s_HubGeneModuleUMAPPlot.png", OUT), widt=1600, height=1600)
ModuleUMAPPlot(
  seurat_obj,
  edge.alpha=0.25,
  sample_edges=TRUE,
  edge_prop=0.1, # proportion of edges to sample (20% here)
  label_hubs=2 ,# how many hub genes to plot per module?
  keep_grey_edges=FALSE
)
dev.off()

Selecting by kME_betacells_allgenes_ND_25neighbors-M1
Selecting by kME_betacells_allgenes_ND_25neighbors-M2
Selecting by kME_betacells_allgenes_ND_25neighbors-M3
Selecting by kME_betacells_allgenes_ND_25neighbors-M4
Selecting by kME_betacells_allgenes_ND_25neighbors-M5


[1] "hub labels"
 [1] "TRERF1"   "EPB41L1"  "KCNH7"    "UNC5D"    "ETV1"     "ROR2"    
 [7] "PDLIM5"   "ARHGEF28" "SGCD"     "HS3ST5"  
NULL
 [1] "TRERF1"   "EPB41L1"  "KCNH7"    "UNC5D"    "ETV1"     "ROR2"    
 [7] "PDLIM5"   "ARHGEF28" "SGCD"     "HS3ST5"  
[1] 327500      3
[1] 6550    4
[1] "making net"
# A tibble: 6 × 5
# Groups:   color [4]
  Var1    Var2    value color     color_alpha
  <fct>   <fct>   <dbl> <chr>     <chr>      
1 CADM1   CADM1       0 blue      #0000FF00  
2 EFNA5   EFNA5       0 blue      #0000FF00  
3 ETV1    ETV1        0 brown     #A52A2A00  
4 ADAMTS2 ADAMTS2     0 turquoise #40E0D000  
5 UBAP2L  UBAP2L      0 turquoise #40E0D000  
6 ADCY2   ADCY2       0 yellow    #FFFF0000  
            gene_name                               module     color   kME_grey
LINC01128   LINC01128 betacells_allgenes_ND_25neighbors-M1 turquoise 0.12696136
SAMD11         SAMD11 betacells_allgenes_ND_25neighbors-M1 turquoise 0.13351788
NOC2L           NOC2L betacells_allgenes_

png 
  2

---

# Scratch